# Module d'analyse de structure
Ce document présente les méthodes d'analyse de la structure des objets Dataset à partir d'un exemple. 
1. Structure de données
2. Fonctions d'analyse
3. Exemple d'implémentation
4. Fonctions de modification de structure
5. Exemple complet

La classe Observation est dérivée de la classe Dataset, ces méthodes sont donc aussi applicables aux objets Observation.

----


## 1 - Structure de données 
### 1.1 - Objet Dataset
L'objet Dataset a deux attributs principaux:
- 'lindex' : list contenant les objets Field qui composent l'objet Dataset
- 'name' : utilisé uniquement par la classe fille 'Observation'

L'instance 'example' ci-dessous contient 4 Field. 

In [1]:
from observation import Sdataset, Ndataset, Sfield, Nfield
from pprint import pprint

example = Sdataset.ntv([[  1,   2,   3,   4,   5,   6],
                        ['a', 'a', 'b', 'b', 'c', 'c'],
                        [100, 100, 200, 200, 300, 300],
                        [True, False, True, False, True, False]])
print(example)
pprint(vars(example))

variables :
    {'i0': [1, 2, 3, 4, 5, 6]}
index :
    {'i1': ['a', 'a', 'b', 'b', 'c', 'c']}
    {'i2': [100, 100, 200, 200, 300, 300]}
    {'i3': [True, False, True, False, True, False]}

{'field': <class 'observation.fields.Sfield'>,
 'lindex': [Sfield[6], Sfield[6], Sfield[6], Sfield[6]],
 'name': 'Sdataset'}


### 1.2 - Objet Field
L'objet Field a trois attributs :
- name : nom
- _codec : liste de valeurs présentes (le rang de la valeur est utilisé pour ordonner les valeurs)
- _keys : liste des rangs des valeurs données lors de la création

Dans l'exemple idx1 ci-dessous, on a choisi de garder dans 'codec' uniquement les valeurs distinctes (default codec)

*Nota : Les valeurs dans cet exemple sont restreintes à des valeurs numériques, chaînes de caractères ou booléens mais elles peuvent également correspondre à des objets de plus haut niveau*

In [2]:
idx0, idx1, idx2, idx3 = example.lindex

print(idx1)
pprint(vars(idx1))

{'i1': ['a', 'a', 'b', 'b', 'c', 'c']}
{'_codec': ['a', 'b', 'c'], '_keys': [0, 0, 1, 1, 2, 2], 'name': 'i1'}


On peut également choisir d'avoir un codec 
- avec toutes les valeurs (full codec)
- avec certaines valeurs dupliquées ou non 

cf ci-dessous les deux exemples.

In [3]:
idx1.tostdcodec(inplace=True)
pprint(vars(idx1), width=180)
idx1.setkeys([0,0,1,2,3,3])
pprint(vars(idx1))

{'_codec': ['a', 'a', 'b', 'b', 'c', 'c'], '_keys': [0, 1, 2, 3, 4, 5], 'name': 'i1'}
{'_codec': ['a', 'b', 'b', 'c'], '_keys': [0, 0, 1, 2, 3, 3], 'name': 'i1'}


## 2 - fonctions d'analyse
Les fonctions d'analyse permettent de mesurer les relations de couplage entre Field et de qualifier la structure Dataset intégrant ces Field.

Les fonctions d'ajustement de la structure (modification) ne sont pas décrites ici.

### 2.1 - Field


<img src="https://loco-philippe.github.io/ES/ilist_index_category.png" width="800">

La méthode infos calcule les valeurs ci-dessous à partir de l'attribut codec conformément au schéma ci-dessus.

In [4]:
idx1.infos

{'lencodec': 4,
 'mincodec': 3,
 'maxcodec': 6,
 'typecodec': 'mixed',
 'ratecodec': 0.6666666666666666}

### 2.2 - couplage entre deux Field
Le couplage entre deux Field est calculé à partir de l'attribut 'keys' conformément au schéma ci-dessous.

<img src="https://loco-philippe.github.io/ES/ilist_link_category.png" width="800">

La méthode 'couplinginfos' calcule les valeurs de couplage (cf exemples ci-dessous)

In [5]:
idx1.reindex() # les attributs 'keys' et 'codec' sont réinitialisées avec le "default codec"
print(idx1.couplinginfos(idx0))
print(idx1.couplinginfos(idx2))
print(idx1.couplinginfos(idx3))

{'dist': 6, 'dmin': 6, 'dmax': 18, 'diff': 3, 'dran': 12, 'rateder': 0.0, 'distomin': 0, 'distomax': 12, 'distance': 3, 'ratecpl': 0.2, 'typecoupl': 'derived'}
{'dist': 3, 'dmin': 3, 'dmax': 9, 'diff': 0, 'dran': 6, 'rateder': 0.0, 'distomin': 0, 'distomax': 6, 'distance': 0, 'ratecpl': 0.0, 'typecoupl': 'coupled'}
{'dist': 6, 'dmin': 3, 'dmax': 6, 'diff': 1, 'dran': 3, 'rateder': 1.0, 'distomin': 3, 'distomax': 0, 'distance': 4, 'ratecpl': 1.0, 'typecoupl': 'crossed'}


### 2.3 - Synthèse des couplages pour l'objet Dataset

La méthode 'couplingmatrix' renvoie une matrice contenant les résultats 'couplinginfos' de chaque couple de Field.

In [6]:
analys = example.analysis.to_dict(mode='id', relations=True)
print("Field: 'i3'\n", analys['fields'][3])
print("\nRelationship: ['i0', 'i2']\n", analys['relations'][5])

Field: 'i3'
 {'lencodec': 2, 'mincodec': 2, 'maxcodec': 6, 'id': 'i3', 'ratecodec': 1.0, 'dmincodec': 0, 'dmaxcodec': 4, 'rancodec': 4, 'typecodec': 'default', 'distroot': 4, 'num': 3, 'category': 'derived', 'pdistance': 'root', 'pdistomin': 'root', 'pderived': 'root'}

Relationship: ['i0', 'i2']
 {'dist': 6, 'relation': ['i0', 'i2'], 'typecoupl': 'derived', 'parentchild': True, 'distance': 3, 'distomin': 0, 'distomax': 12, 'distributed': False, 'ratecpl': 0.2, 'rateder': 0.0, 'dmax': 18, 'dmin': 6, 'diff': 3, 'dran': 12}


### 2.4 - Qualification de la structure de l'objet Dataset

Les propriétés indiquées ci-dessous sont calculées par la méthode 'indexinfos'.

<img src="https://loco-philippe.github.io/ES/ilist_properties.png" width="600">

Ceci permet de calculer les informations 'primary', 'dimension'.

Dans l'exemple ci-dessous les Field 'i1' et 'i3' sont de type 'primary', ils forment donc une matrice (dimension 2).

In [7]:
print('dimension : ', example.dimension)
print('\ndefault partition:\n', example.analysis.field_partition(mode='id'))

dimension :  2

default partition:
 {'primary': ['i1', 'i3'], 'secondary': ['i2'], 'unique': [], 'variable': ['i0']}


La distinction entre Field 'variable', Field 'primary' et Field 'secondary' est compatible avec les outils d'analyse de données comme Xarray.
cf exemple ci-dessous :

In [8]:
example.to_xarray()

<xarray.DataArray 'i0' (i1: 3, i3: 2)>
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=object)
Coordinates:
  * i1       (i1) object 'a' 'b' 'c'
  * i3       (i3) object 'true' 'false'
    i2       (i1) object 100 200 300

## 3 - Exemple d'implémentation 
L'exemple ci-dessous présente une implémentation simple pour le calcul des relations de couplage.

In [9]:
def check_relationship(field1, field2):
    dist = len(set(zip(field1, field2)))
    len1 = len(set(field1))
    len2 = len(set(field2))
    
    if dist == len1 and dist > len2:    return "field 2 is derived from field 1"
    if dist == len2 and dist > len1:    return "field 1 is derived from field 2"    
    if dist == len1 and dist == len2:   return "field 2 and field 1 are coupled"
    if dist == len1 * len2:             return "field 2 and field 1 are crossed"
    return "field 1 and field 2 are linked"

example = [ [  'T1',    'T2',   'T2',    'T1',    'T2',   'T1'],
            [ 'jan',   'apr',  'jun',   'feb',   'may',  'jan'],
            ['john',  'paul', 'leah',  'paul',  'paul', 'john'],
            ['jock', 'paulo', 'lili', 'paulo', 'paulo', 'jock'],
            [  2020,    2020,   2021,    2021,    2022,   2022],
            [  's1',    's2',   's1',    's2',    's1',   's2']]

print(check_relationship(example[0], example[1]))  #field 1 is derived from field 2
print(check_relationship(example[2], example[3]))  #field 2 and field 1 are coupled
print(check_relationship(example[4], example[5]))  #field 2 and field 1 are crossed
print(check_relationship(example[1], example[4]))  #field 1 and field 2 are linked

field 1 is derived from field 2
field 2 and field 1 are coupled
field 2 and field 1 are crossed
field 1 and field 2 are linked


## 4 - Fonctions de modification de structure

Deux types de fonctions sont accessibles :
- modification de codification (sans modifier les valeurs externes)

<img src="https://loco-philippe.github.io/ES/ilist_codec_adjustment.png" width="700">

- modification de données (sans modifier la codification)

<img src="https://loco-philippe.github.io/ES/ilist_value_adjustment.png" width="700">

Un exemple d'application de ces fonctions est fourni [ici](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Examples/Iindex/Iindex_structure_analysis.ipynb).

## 5 - Exemple complet
Un exemple complet est fourni sur [ce lien](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/test_IRVE.ipynb). 

Il concerne les bornes IRVE (infrastructures de recharge des véhicules électriques).

Les données d'origine sont disponibles également : [fichier consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv](https://github.com/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv).

Les temps de réponse des opérations concernées sont également indiqués.
